<a href="https://colab.research.google.com/github/tmskss/ManPageSum/blob/main/colab/finetune_falcon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/tmskss/ManPageSum.git
%cd ManPageSum/colab
from install import *
install_requirements()

from utils import *
setup()

Cloning into 'ManPageSum'...
remote: Enumerating objects: 1238, done.
remote: Counting objects: 100% (1238/1238), done.
remote: Compressing objects: 100% (954/954), done.
remote: Total 1238 (delta 310), reused 1197 (delta 283), pack-reused 0
Receiving objects: 100% (1238/1238), 9.25 MiB | 13.84 MiB/s, done.
Resolving deltas: 100% (310/310), done.
/content/ManPageSum/colab
⏳ Installing base requirements ...
✅ Base requirements installed!
✅ Summary requirements installed!
Using transformers v4.32.1
Using datasets v2.0.0


In [34]:
!pip install ludwig
!pip install ludwig[llm]
!pip install bitsandbytes
!pip install accelerate

In [3]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)

def clear_cache():
  if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [4]:
import getpass
import locale; locale.getpreferredencoding = lambda: "UTF-8"
import logging
import os
import torch
import yaml

from ludwig.api import LudwigModel


os.environ["HUGGING_FACE_HUB_TOKEN"] = getpass.getpass("Token:")
assert os.environ["HUGGING_FACE_HUB_TOKEN"]

Token:··········


In [27]:
from datasets import load_dataset

dataset = load_dataset("tmskss/linux-man-pages-tldr-summarized")
print(f"Features: {dataset['train'].column_names}")

  0%|          | 0/1 [00:00<?, ?it/s]

Features: ['Command', 'Text', 'Summary']


In [31]:
from google.colab import data_table; data_table.enable_dataframe_formatter()
import numpy as np; np.random.seed(123)
import pandas as pd

df = pd.DataFrame(dataset['train'])

# We're going to create a new column called `split` where:
# 90% will be assigned a value of 0 -> train set
# 5% will be assigned a value of 1 -> validation set
# 5% will be assigned a value of 2 -> test set
# Calculate the number of rows for each split value
total_rows = len(df)
split_0_count = int(total_rows * 0.9)
split_1_count = int(total_rows * 0.05)
split_2_count = total_rows - split_0_count - split_1_count

# Create an array with split values based on the counts
split_values = np.concatenate([
    np.zeros(split_0_count),
    np.ones(split_1_count),
    np.full(split_2_count, 2)
])

# Shuffle the array to ensure randomness
np.random.shuffle(split_values)

# Add the 'split' column to the DataFrame
df['split'] = split_values
df['split'] = df['split'].astype(int)

In [32]:
df.head(10)

,Command,Text,Summary,split
0,chgrp,The chgrp utility shall set the group ID of t...,# chgrp\n> Change group ownership of files and...,0
1,more,more is a filter for paging through text one ...,"# more\n> Open a file for interactive reading,...",0
2,git-hash-object,Computes the object ID value for an object wi...,# git hash-object\n> Computes the unique hash ...,0
3,id,"If no user operand is provided, the id utilit...",# id\n> Display current user and group identit...,0
4,nl,"Write each FILE to standard output, with line...","# nl\n> A utility for numbering lines, either ...",0
5,git-check-ignore,For each pathname given via the command-line ...,# git check-ignore\n> Analyze and debug Git ig...,0
6,tcpdump,Tcpdump prints out a description of the conte...,# tcpdump\n> Dump traffic on a network. More i...,0
7,users,Output who is currently logged in according t...,# users\n> Display a list of logged in users. ...,0
8,git-rev-list,List commits that are reachable by following ...,# git rev-list\n> List revisions (commits) in ...,0
9,lpr,lpr submits files for printing. Files named o...,# lpr\n> CUPS tool for printing files. See als...,0


In [36]:
import accelerate
import bitsandbytes

zero_shot_config = yaml.safe_load(
  """
  model_type: llm
  base_model: tiiuae/falcon-7b

  input_features:
    - name: Text
      type: text

  output_features:
    - name: Summary
      type: text

  prompt:
    template: >-
      Below is a text describing a linux command. Summarize it.

      ### Instruction: {Text}

      ### Response:

  generation:
    temperature: 0.1 # Temperature is used to control the randomness of predictions.
    max_new_tokens: 2048

  preprocessing:
    split:
      type: fixed

  quantization:
    bits: 4
  """
)

model = LudwigModel(config=zero_shot_config, logging_level=logging.INFO)
results = model.train(dataset=df, trust_remote_code = True)

Loading tiiuae/falcon-7b requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/tiiuae/falcon-7b. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y
Loading tiiuae/falcon-7b requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/tiiuae/falcon-7b. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y


INFO:ludwig.utils.print_utils:
INFO:ludwig.utils.print_utils:╒════════════════════════╕
INFO:ludwig.utils.print_utils:│ EXPERIMENT DESCRIPTION │
INFO:ludwig.utils.print_utils:╘════════════════════════╛
INFO:ludwig.utils.print_utils:
INFO:ludwig.api:╒══════════════════╤═════════════════════════════════════════════════════════════════════════════════════════╕
│ Experiment name  │ api_experiment                                                                          │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ Model name       │ run                                                                                     │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ Output directory │ /content/ManPageSum/colab/results/api_experiment_run_4                                  │
├──────────────────┼─────────────────────────────────────────────────────────────────

Loading tiiuae/falcon-7b requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/tiiuae/falcon-7b. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y


INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of tiiuae/falcon-7b tokenizer


Loading tiiuae/falcon-7b requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/tiiuae/falcon-7b. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y


...

INFO:ludwig.data.preprocessing:Building dataset: DONE
INFO:ludwig.data.cache.manager:Writing preprocessed training set cache to /content/ManPageSum/colab/e5855d104bdc11eea6640242ac1c000c.training.hdf5
INFO:ludwig.data.cache.manager:Writing preprocessed validation set cache to /content/ManPageSum/colab/e5855d104bdc11eea6640242ac1c000c.validation.hdf5
INFO:ludwig.data.cache.manager:Writing preprocessed test set cache to /content/ManPageSum/colab/e5855d104bdc11eea6640242ac1c000c.test.hdf5
INFO:ludwig.data.cache.manager:Writing train set metadata to /content/ManPageSum/colab/e5855d104bdc11eea6640242ac1c000c.meta.json
INFO:ludwig.api:
Dataset Statistics
INFO:ludwig.api:╒════════════╤═══════════════╤════════════════════╕
│ Dataset    │   Size (Rows) │ Size (In Memory)   │
╞════════════╪═══════════════╪════════════════════╡
│ Training   │           432 │ 101.38 Kb          │
├────────────┼───────────────┼────────────────────┤
│ Validation │            24 │ 5.75 Kb            │
├────────────┼─

Loading tiiuae/falcon-7b requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/tiiuae/falcon-7b. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y


INFO:ludwig.models.llm:Loading large language model...


Loading tiiuae/falcon-7b requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/tiiuae/falcon-7b. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y


ImportError: ignored